#### [PyTorch를 활용한 머신러닝, 딥러닝 철저 입문]
## [Chapter 6-4] 예제: 뉴스 기사 분류

---

## \#0. Download Data

다음 주소에서 데이터를 다운로드 한다.

> http://kristalinfo.com/TestCollections/#hkib

이 중 `HKIB-20000` 폴더에 있는 데이터만을 사용한다. 

---

## \#1. Import Modules

In [1]:
import os
import re

from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from konlpy.tag import Hannanum
from konlpy.tag import Kkma

import pandas as pd
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

---

## \#2. Conversion : Raw data → Article text files

### 2-1. Directory Setting

- raw data가 담겨있는 디렉토리 주소를 가져온다.

In [2]:
dir_prefix = './data/'
target_dir = 'HKIB-20000'

- data를 가공해서 정리된 형태의 파일들을 저장할 디렉토리를 지정한다.

In [3]:
saving_dir = 'articles'

### 2-2. Classification categories

- 기사들을 분류할 category를 설정한다.


- category는 2007년 (`#CAT'07`) 의 대분류 9가지로 설정한다.

In [4]:
category_dirs = ['health', 'economy', 'science', 'education', 'culture', \
                 'society', 'industry', 'leisure', 'politics']
category_dirs

['health',
 'economy',
 'science',
 'education',
 'culture',
 'society',
 'industry',
 'leisure',
 'politics']

- category를 추출할 prefix들을 저장한다.

In [5]:
category_prefixes = ['건강', '경제', '과학', '교육', '문화', '사회', '산업', '여가', '정치']
category_prefixes

['건강', '경제', '과학', '교육', '문화', '사회', '산업', '여가', '정치']

### 2-3. Articles Preprocessing & Saving

- raw data로 쓰일 모든 file을 디렉토리 주소를 따라 가져와서 변수에 저장한다.

In [6]:
files = os.listdir(dir_prefix + target_dir)

- raw data file들을 전처리한다.

In [7]:
%%time
# file명(str)을 순차적으로 가져오기
for file in files:

    # 'txt'파일만 처리
    if not file.endswith('.txt'):
        continue
    
    
    # text file별로 하나씩 열어서 처리
    with open(dir_prefix + target_dir + '/' + file) as cur_file:
        
        # 변수 초기화
        doc_count = 0
        docs = []
        cur_doc = None
        
        
        # file의 각 줄을 scan하면서 docs에 기사별로 리스트 생성
        for cur_line in cur_file:

            # 기사의 첫 줄은 '@DOCUMENT' : 첫 줄 처리
            if cur_line.startswith('@DOCUMENT'):
                # 새로운 기사가 시작됐을 때, 이전 기사 전체를 docs에 추가한다
                if cur_doc is not None:
                    docs.append(cur_doc)
                # 그 후 cur_doc은 새 기사의 첫 줄로 초기화
                cur_doc = cur_line
                doc_count += 1
                continue
                
            # 첫줄 외에는 cur_line을 한 줄씩 모두 더해나간다
            cur_doc += cur_line
            
 
        # docs의 각 기사를 대주제별로 분류
        for doc in docs:
            
            # 한 기사를 한 줄씩 나눠서 리스트 생성
            doc_lines = doc.split("\n")

            # "#DocID : " 뒤의 숫자
            doc_no = doc_lines[1][9:]
            
            # "#CAT'03: " 뒤의 카테고리 추출
            doc_cat03 = ''
            for line in doc_lines[:10]:
                if line.startswith("#CAT'07:"):
                    doc_cat03 = line[10:]
                    break
            
            # 추출한 주제별로 디렉토리 정리
            for category_prefix in category_prefixes:
                if doc_cat03.startswith(category_prefix):
                    dir_index = category_prefixes.index(category_prefix)
                    break
                    
            # 문서 정보는 제거하고 기사 본문만 저장
            filtered_lines = []
            for line in doc_lines:
                if not (line.startswith("#") or line.startswith("@") or line.startswith("<")):
                    line = line.strip()
                    if line.strip():
                        filtered_lines.append(line)
                
                
            # 주제별 디렉토리에 기사를 파일로 저장
            filename = 'hkib-' + doc_no + '.txt'
            filepath = dir_prefix + target_dir + '/' + saving_dir + '/' + category_dirs[dir_index]
            
            # 디렉토리가 없으면 폴더 생성
            if not os.path.exists(filepath):
                os.makedirs(filepath)
            
            # filtered_lines의 기사본문을 파일로 저장
            f = open(filepath + '/' + filename, 'w')
            f.write(' '.join(filtered_lines))
            f.close()

CPU times: user 2.57 s, sys: 2.98 s, total: 5.54 s
Wall time: 5.78 s


### \# Jeina's comment
- **책에서 사용한 카테고리는 2003년 카테고리 (`#CAT'03`)으로, 총 8가지의 카테고리 (정치 제외)가 사용되었다.**  
    : 본 코드에서는 정치까지 포함한 2007년 형식의 카테고리를 사용해서 파일을 저장하였다.

### 2-4. Show the sample saved file

저장이 잘 되었는지 `culture` 폴더의 한 기사만 확인을 해보자.

In [8]:
culture_files = os.listdir(dir_prefix + target_dir + '/' + saving_dir + '/culture')

for file in culture_files:
    print("[ filename: ", file, ']\n')
    with open(dir_prefix + target_dir + '/' + saving_dir + '/culture/' + file) as culture_article:
        for line in culture_article:
            print(line, end="")
    break

[ filename:  hkib-12549.txt ]

조계사 폭력사태를 수사중인 서울경찰청 특별수사본부(본부장 서정옥형사 부장)는 15일 총무원측이 폭력배동원 대가로 3천만원의 자금이 지급한 사실이 확인됨에 따라 이자금의 출처에 대한 수사를 강화하고 있다. 경찰은 구속중인 무성승려(31)가 자금출처에 대해 함구하고 있으나 액수로 보아총무원 수뇌부의 지시나 결재없이는 지출이 어렵다는 현실적인 판단에 따라 조사결과에 따라서는 서의현 전원장을 비롯한 구총무원수뇌부의 소환, 조사도 불가피할 것으로 보인다. 경찰은 또 구속중인 불출이파 두목 반봉환씨(32)가 폭력배 동원 대가로 돈을 지급한 것이외에 지방사찰의 이권을 주겠다고 약속했다고 진술한 점도 구총무원 수뇌부의 개입혐의를 입증해 주는 중요한 단서로 보고 있다. 경찰은 이와 함께 폭력배 동원자금의 출처를 캐기 위해 무성승려로부터 3천만원중 1천8백만원을 1백만원권 자기앞수표로 받았다는 김금남씨(29. 구속중)의 진술에 따라 수표추적 작업에 들어가는 한편 구총무원 예금 계좌를 압수수색하는 방안도 검토중이다. 경찰은 또 이번 사건에 동원된 폭력배들이 3백여명인 것이 확인됨에 따라 김씨와 반씨 등을 상대로 자세한 폭력배 동원경위를 캐고 있다. 경찰은 이날 폭력가담자 박성경씨(29.무직.서울 동대문구 장안동 장안 아파트 103동 102호)에 대해 폭력행위등 처벌에 관한 법률 위반 혐의로 구속영장을 신청했다. 한편 경찰은 이날까지 폭력가담자 43명을 검거해 22명을 구속했으며 민병돈씨(29.서울 송파구 방이동 153의 2) 등 신원이 확인된 14명의 폭력용의자들을 쫓고 있다.

> 기사의 본문만 깔끔하게 잘 저장이 되었음을 확인할 수 있다.

---

> **파일이 저장되어있는 상태라면 여기서부터 돌리기**

---

## \#3. Data Preprocessing : using `RegEx`

- 이제 기사 별로 파일 하나씩 잘 저장되어있는 데이터에서, `RegEx`를 이용해서 불필요한 문자를 모두 제거하고 최대한 한글만 남겨보도록 하자.

### 3-1. RegEx Filtering Function

- raw text data를 받으면 RegEx로 필터링 하는 함수를 생성한다.

In [9]:
def regex_filter(raw):
    reg_raw = re.sub(r'[0-9a-zA-Z]', "", raw)
    reg_raw = re.sub(r'[-@#:/◆<>!-"*「」『』:~=%㎜㎴㎏·`【】…“~☎▲■]', "", reg_raw)
    reg_raw = re.sub(r'[\'\(\)]', "", reg_raw)
    reg_raw = re.sub(r'[\[\]]', "", reg_raw)
    reg_raw = re.sub(r'[ ]+', " ", reg_raw)
    reg_raw = re.sub(r'[.]+', ".", reg_raw)
    
    return reg_raw

- 먼저 한 기사를 이용해서 RegEx filtering이 잘 적용되는지 확인해보자.

### 3-2. RegEx Filtering Test

In [10]:
files = os.listdir(dir_prefix + target_dir + '/' + saving_dir + '/' + 'health')


# 원래 저장되어있는 raw한 기사 텍스트 출력
for file in files:
    print("[ filename: ", file, ']\n')
    print('< BEFORE filtered >\n')

    f = open(dir_prefix + target_dir + '/' + saving_dir + '/' + 'health' + '/' + file, \
             'r', encoding='utf-8')

    for line in f:
        print(line, end="")
    break

print('\n\n')


# regex_filter 함수를 이용해서 필터링한 text 출력
for file in files:
    print("< AFTER filtered with RegEx > \n")

    f = open(dir_prefix + target_dir + '/' + saving_dir + '/' + 'health' + '/' + file, \
             'r', encoding='utf-8')

    raw = f.read()
    reg_raw = regex_filter(raw)
    print(reg_raw, '\n')

    f.close()
    break

[ filename:  hkib-94076.txt ]

< BEFORE filtered >

한국인백색증의 돌연변이가 발견됨으로써 출산전에 태아가 백색증돌연변이 를 갖고 있는지 조사할 수 있게 됐다. 서울대의대 박경찬교수(피부과)팀은 최근 백색증을 일으키는 돌연변이 유전자검사법을 개발, 3명의 환자로부터 3종류 6개의 유전자돌연변이를 발견했다고 밝혔다. 3종류의 유전자돌연변이는 아미노산치환유전자이상이 2종, 아미노산 구조이동유전자이상이 1종으로 밝혀졌다. 이 팀이 개발한 유전자검색법은 SSCP(단섬유구조다양성)기술을 이용, 이중나선구조를 가진 DNA와 단선구조의 RNA를 쉽게 검색할 수 있도록 단선구조로 변형해 검색하는 방법이다. 이에 따라 임신 16~17주정도에 양수검사로 세포중 DNA를 추출, PCR(중합 효소반응검사)법과 함께 이 검사법을 실시하면 출산전에 돌연변이여부를 확인할 수 있게 됐다. 종전의 검사법은 여러가지 실험등 임상에서 많은 환자들을 대상으로 실시 하기엔 어려움이 있어 새로운 검사법을 연구하게 됐다고 박교수는 말했다. 백색증은 피부 모발및 안구의 색소가 없어지는 질환으로 피부및 안구의 색소가 부족해 광선에 예민, 피부암의 발생빈도가 높다. 시력장애도 심각해 정상적인 사회활동에 장애가 있다. (한국경제신문 1994년 10월 30일자). DNA 개발 검색 고충 관련법 광선 기술 단섬유 대상 모발 발견 방법 백색증 변형 부족 서울대학 세포 실시 실험 아미노산 양수검사 연구 유전자 의과대학 의료 의료계소식 이용 임신 장애 정상 조사 질병 추출 태아 피부 피부암 한국인 확인 환자


< AFTER filtered with RegEx > 

한국인백색증의 돌연변이가 발견됨으로써 출산전에 태아가 백색증돌연변이 를 갖고 있는지 조사할 수 있게 됐다. 서울대의대 박경찬교수피부과팀은 최근 백색증을 일으키는 돌연변이 유전자검사법을 개발, 명의 환자로부터 종류 개의 유전자돌연변이를 발견했다고 밝혔다. 종류의 유전자돌연변이는 아미노산치환유전자이상이 종, 아미노산 

> 원래 특수기호 및 숫자, 알파벳이 섞여있었던 데이터에서 `RegEx`를 사용한 후 모두 제거되었음을 확인할 수 있다.

---

## \#4. Get `X` / `y` data by tokenizing in `noun`

- 이제 모두 저장된 기사 데이터에서 학습에 활용할 `X`, `y` Dataset을 생성해보자.

### 4-1. Select Categories

In [11]:
category_dirs

['health',
 'economy',
 'science',
 'education',
 'culture',
 'society',
 'industry',
 'leisure',
 'politics']

- category는 위의 총 9가지로 이루어져 있다. 
- 이 중 시간관계상 기사가 적당히 들어있는 (데이터 크기가 2MB를 넘지 않는)  
[ `education`, `health`, `leisure`, `politics`, `science` ]   
의 5가지만 분류를 해보도록 한다.

In [12]:
dirs = ['education', 'health', 'leisure', 'politics', 'science']
dirs

['education', 'health', 'leisure', 'politics', 'science']

### 4-2. Tokenize by noun

- `X`, `y`의 빈 리스트를 생성한다.

In [13]:
X = list()
y = list()

- tokenizer 로는 꼬꼬마 라이브러리 (`Kkma`)를 사용한다.

In [14]:
tokenizer = Kkma()

- 이제 각 디렉토리의 모든 파일을 tokenize 한다.

In [15]:
%%time
for idx, directory in enumerate(dirs):
    print("directory [{}] is now being tokenized. . .".format(directory))
    
    # directory별로 file을 연다
    files = os.listdir(dir_prefix + target_dir + '/' + saving_dir + '/' + directory)
    
    # 각 text file 별로 tikenizing 진행
    for file in files:
        f = open(dir_prefix + target_dir + '/' + saving_dir + '/' + directory + '/' + file, 'r', encoding='utf-8')
        
        # regex_filter로 전처리
        raw = f.read()
        reg_raw = regex_filter(raw)
        
        # tokenizer로 분해된 tokens 리스트 생성
        tokens = tokenizer.nouns(reg_raw)
        tokens = [token for token in tokens]
        
        # noun으로만 분해된 기사를 X에 추가
        nouns = " ".join(tokens)
        X.append(nouns)
        
        # y에는 label 추가
        y.append(idx)
        
        f.close()

directory [education] is now being tokenized. . .
directory [health] is now being tokenized. . .
directory [leisure] is now being tokenized. . .
directory [politics] is now being tokenized. . .
directory [science] is now being tokenized. . .
CPU times: user 29min 53s, sys: 5.27 s, total: 29min 59s
Wall time: 10min 54s


> 총 `5.13MB`의 데이터를 처리하는 데에 `10분 43초`나 걸렸다 !

#### - `X`와 `y`의 개수 확인

In [16]:
len(X), len(y)

(2521, 2521)

둘 다 2521개의 기사가 잘 담겼음을 확인

#### 여기서 `y` 값에 따른 카테고리는 다음과 같다.

In [17]:
print('category', '\t', 'label')
print('-----------', '\t', '-----')

for idx, category in enumerate(dirs):
    print('{}  \t {}'.format(category, idx))

category 	 label
----------- 	 -----
education  	 0
health  	 1
leisure  	 2
politics  	 3
science  	 4


### 4-3. Get DataFrame

- 생성된 `X`와 `y`를 바탕으로 DataFrame을 생성해보자.

In [18]:
df = pd.DataFrame(X, columns=['article'])
df['label'] = y
print(df.shape)
df.tail()

(2521, 2)


,article,label
2516,럭키 럭키대표 대표 성재 성재갑 갑 최근 아마 틱 우 우디 디 무스크향 첨가 봉 캐...,4
2517,식물 식물세포 세포 실험실 실험실쥐 쥐 이식 정착 획기적 과학 과학실험 실험 성공 ...,4
2518,럭키 럭키대표 대표 성재 성재갑 갑 그동안 전량 전량수입 수입 의존 온 옥외 옥외광...,4
2519,경부 경부고속철도 고속 철도 건설 건설사업 사업 차 구간 충북 청원 청원군 군 강외...,4
2520,중부 해안 황사 비상 서해안 서해안지방 지방 예년 달 발생 기상 보건 보건당구 당구...,4


#### - 첫 번째 기사를 확인해보자.

In [19]:
df['article'][0]

'정부 일 충북 충북대 대 신임 총장 대학 이랑 이랑호 호 학원장 임명 등 개 국립대 인사 단행 부산 부산수산대 수산 장 장선덕 선덕 부산수산해양대학 해양 목 목포대 포대 이태 이태근 근 대학원 삼척 삼척산업대학 산업 김영달 교수 대구 대구교육대 교육대 노 노정식 정식 교 교수가 수가 공주 공주교육대 문 문낙진 낙진'

명사로만 이루어져 있는 기사가 잘 만들어져 있음을 확인!

---

## \#5. TF-IDF

- 생성된 X, y의 데이터셋을 바탕으로 각 기사와 단어들간의 관계를 나타내고 단어의 가중치를 부여하는 TF-IDF를 생성한다.

### 5-1. TF DataFrame 생성

In [20]:
# numpy array로 변환
X = np.array(X)
y = np.array(y)

In [21]:
# CountVectorizer로 객체 생성
count_vectorizer = CountVectorizer()

In [22]:
X_cntvecs = count_vectorizer.fit_transform(X)
x_cntarray = X_cntvecs.toarray()

# TF(term frequency) 데이터 프레임 생성
TF = pd.DataFrame(x_cntarray)
print(TF.shape)
TF.tail()

(2521, 69941)


,0,1,2,3,4,5,6,7,8,9,...,69931,69932,69933,69934,69935,69936,69937,69938,69939,69940
2516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- 총 2521개의 기사에는 69941개의 unique한 명사들로 구성되어있다.

#### 각 명사들은 어떤 단어들인지 확인해보자.

In [23]:
for word, idx in sorted(count_vectorizer.vocabulary_.items(), key=lambda x: x[1])[:10]:
    print(idx, '\t', word)
print("...")
for word, idx in sorted(count_vectorizer.vocabulary_.items(), key=lambda x: x[1])[-10:]:
    print(idx, '\t', word)

0 	 ㄴ목
1 	 ㆍ과
2 	 가가
3 	 가가린
4 	 가가치
5 	 가가치산업
6 	 가감
7 	 가감리
8 	 가건물
9 	 가검물
...
69931 	 힌두교도
69932 	 힌두교도들이
69933 	 힌두지가
69934 	 힐데스하임
69935 	 힐러리
69936 	 힐사가
69937 	 힐호텔
69938 	 힘겨루기
69939 	 힘라
69940 	 힘쓰겠


- ㄱ부터 ㅎ까지 알파벳 순으로 여러가지 단어들이 들어있음을 확인

### 5-2. TF-IDF DataFrame

- 이제 각 단어들의 TF-IDF를 계산해보자.

In [24]:
# TFIDF 객체 생성
tfidf_vectorizer = TfidfVectorizer(use_idf=True)

In [25]:
X_tfidf_vecs = tfidf_vectorizer.fit_transform(X)
X_tfidf_array = X_tfidf_vecs.toarray()

# TF-IDF (inverse document frequency) DataFrame 생성
TFIDF = pd.DataFrame(X_tfidf_array)
print(TFIDF.shape)
TFIDF.tail()

(2521, 69941)


,0,1,2,3,4,5,6,7,8,9,...,69931,69932,69933,69934,69935,69936,69937,69938,69939,69940
2516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 이제 각 단어들의 가중치가 반영된 TF-IDF 데이터 프레임이 생성되었다.

---

## \#6. Get Train / Test Datasets

- `train_test_split` 함수로 Dataset을 split한다.

In [26]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(X_tfidf_array, y, test_size=0.2)

print(len(train_X), len(test_X))
print(len(train_y), len(test_y))

2016 505
2016 505


총 2521개의 기사는 train과 test dataset에 각각 2016, 505개로 분할되었다.

---

## \#7. Make Tensor

### 7-1. Transform Data Type

#### 완성된 train, test 데이터를 PyTorch의 `tensor 데이터`로 변환한다.

In [27]:
# train data
train_X = torch.from_numpy(train_X).float()
train_y = torch.from_numpy(train_y).long()

# test data
test_X = torch.from_numpy(test_X).float()
test_y = torch.from_numpy(test_y).long()

train_X.shape, train_y.shape

(torch.Size([2016, 69941]), torch.Size([2016]))

#### train의 X와 y데이터를 `tensorDataset` 으로 합친다.

In [28]:
train = TensorDataset(train_X, train_y)
train[0]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor(2))

### 7-2. Mini Batch

#### train 데이터를 미니배치로 학습시킬 수 있도록 16개 단위로 분할한다.

In [29]:
train_loader = DataLoader(train, batch_size=200, shuffle=True)

---

## \#8. Neural Network Composition

### 8-1. Neural Network class

이번에는 중간층이 총 5층인 신경망을 구성한다.
- 입력층의 노드 수 : `69941개` [`x`의 feature 수, 즉 단어의 개수]
- 중간층의 노드 수 : 각 `128 ~ 1024개`
- 출력층의 노드 수 : `5개` [`y`의 target, 즉 카테고리의 개수]

In [30]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # 입력층
        self.func0 = nn.Linear(69941, 512)
        
        # 중간층(5층)
        self.func1 = nn.Linear(512, 256)
        self.func2 = nn.Linear(256, 256)
        self.func3 = nn.Linear(256, 128)
        self.func4 = nn.Linear(128, 128)
        
        # 출력층
        self.func_out = nn.Linear(128, 5)
        
        
    def forward(self, x):
        # 입력층 ~ 중간층(4층) : 활성화함수는 relu 함수 사용
        x = F.relu(self.func0(x))
        x = F.relu(self.func1(x))
        x = F.relu(self.func2(x))
        x = F.relu(self.func3(x))
        x = F.relu(self.func4(x))
        
        # 출력층
        x = self.func_out(x)
        
        # 출력층 활성화함수 : log_softmax
        y = F.log_softmax(x, dim=1)
        
        return y

### 8-2. NN Instance 생성

In [31]:
model = Net()
model

Net(
  (func0): Linear(in_features=69941, out_features=512, bias=True)
  (func1): Linear(in_features=512, out_features=256, bias=True)
  (func2): Linear(in_features=256, out_features=256, bias=True)
  (func3): Linear(in_features=256, out_features=128, bias=True)
  (func4): Linear(in_features=128, out_features=128, bias=True)
  (func_out): Linear(in_features=128, out_features=5, bias=True)
)

입력층과 중간층 4층, 그리고 출력층까지 확인

---

## \#9. Model Training

### 9-1. Loss Function

In [32]:
criterion = nn.CrossEntropyLoss()

### 9-2. Optimizer

In [33]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

### 9-3. Training

In [34]:
%%time
print("{} \t {}".format("iteration", "total loss"))
print("--------- \t ----------------------")

for i in range(50):
    total_loss = 0
    
    for train_X, train_y in train_loader:
        
        # 계산 그래프 구성
        train_X, train_y = Variable(train_X), Variable(train_y)
        
        # gradient 초기화
        optimizer.zero_grad()
        
        # forward 계산
        output = model.forward(train_X)
        
        # loss 계산
        loss = criterion(output, train_y)
        
        # 오차 역전파
        loss.backward()
        
        # gradient update
        optimizer.step()
        
        # 누적 오차 계산
        total_loss += loss.data
        
    # 5회마다 loss 출력 (1 epoch=10회)
    if i % 5 == 0:
        if i % 10 == 0:
            print("{} \t\t {} \t ({} epoch)".format(i, total_loss, i//10))
        else:
            print("{} \t\t {} \t".format(i, total_loss))

iteration 	 total loss
--------- 	 ----------------------
0 		 15.973578453063965 	 (0 epoch)
5 		 0.9567501544952393 	
10 		 0.2107725888490677 	 (1 epoch)
15 		 0.1800481528043747 	
20 		 0.15681304037570953 	 (2 epoch)
25 		 0.22247055172920227 	
30 		 0.23536035418510437 	 (3 epoch)
35 		 0.14417190849781036 	
40 		 0.13552722334861755 	 (4 epoch)
45 		 0.1299942433834076 	
CPU times: user 9min 42s, sys: 49.3 s, total: 10min 32s
Wall time: 7min 27s


### Jeina's Comment
#### loss가 수렴해나가는 것에 있어서 layer의 개수와 node의 개수가 critical하다!
- layer 개수를 4~7개정도로 늘였다 줄여보고, 각 층의 node개수도 128~1024까지 여러가지로 바꾸면서 해봄
- 책에서는 category를 2개만 사용하고, 또한 기사의 개수, 단어의 개수가 각각 1000개, 30000개 정도였음
- 하지만 나는 5개의 category를 분류하는 것을 시도해봤으므로 기사와 단어의 개수가 **두 배정도** 늘어남
- 직관적으로 layer의 node개수를 두 배 정도 늘려봤고, 여러가지 시도를 해본 결과 node의 수에 따라 수렴 여부가 크게 달라짐
- 예를 들어 hidden layer 1층의 node를 1024개까지 늘렸을 때 parameter가 너무 많은 까닭인지 전혀 수렴하지 않음
- 또한 layer를 그저 6-7층까지 올렸을 때에도 전혀 수렴하지 않음
- 512개의 노드로 시작해서 128개 까지 적당히 줄였을 때 가장 이상적으로 수렴하는 것을 확인

---

## \#10. Accuracy

### 10-1. test data 로 accuracy 계산

In [35]:
# 계산 그래프 구성
test_X, test_y = Variable(test_X), Variable(test_y)

# 결과를 0 또는 1이 되도록 변환
result = torch.max(model(test_X).data, 1)[1]

# test_y 와 결과가 같은 예측값의 개수 (맞춘 개수) 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

accuracy

0.7227722772277227

정확도는 약 72.3% 정도.. 딱히 높지 않은 듯 하다. 이유는?